# Generate genelist for NIAGADS table1

In [1]:
library(data.table)
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::between()     masks data.table::between()
x dplyr::filter()      masks stats::filter()
x dplyr::first()       masks data.table::first()
x lubridate::hour()    masks data.table::hour()
x lubridate::isoweek() masks data.table::isoweek()
x dplyr::lag()         masks stats::lag()
x dplyr::last()        masks data.table::last()
x lubridate::mday()    masks data.table::mday()
x lubridate::minute()  masks data.table::minute()
x lubridate::month()   masks data.table::month()
x lubridate::quarter() masks data.table::quarter()
x lubridate::second()  masks data.table::second()
x purrr::transpose()   masks data.table::transpose()
x lubridate::wday() 

## load faltten table and gene reference 

In [2]:
fla_tb <- fread('/data/analysis_result/finemapping_twas/gwas_xqlt_overlap/20240418/Fungen_xQTL_allQTL.overlapped.gwas.export.snATAC.ADGWAS_fix.csv.gz')

In [5]:
fla_tb %>% head(n=2)

variant_id,chr,pos,pip,maf,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,cs_coverage_0.95_min_corr,cs_coverage_0.7_min_corr,cs_coverage_0.5_min_corr,annotated_susie_cs,study,region,method,betahat,sebetahat
<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>
chr10:15118:G:A,chr10,15118,0.9961109,0.01265823,5,5,5,5,5,5,0,ROSMAP_Astrocyte_snATACQTL,chr10_0_4137718,NA,NA,NA
chr10:1602399:G:A,chr10,1602399,0.9979509,0.04166667,15,15,15,15,15,15,0,ROSMAP_Astrocyte_snATACQTL,chr10_0_4137718,NA,NA,NA


In [43]:
gene_ref <- fread('/data/resource/References/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list')

### clean up flatten table to xQTL only

In [143]:
fla_tb_95_dis <- fla_tb %>% filter(cs_coverage_0.5_min_corr	 > 0, !(str_detect(study, '^AD'))) %>% select(study, region, chr, pos)  %>% merge(., gene_ref, by.x = 'region', by.y = 'gene_id')

In [144]:
fla_tb_95_dis_noQTL<- fla_tb %>% filter(cs_coverage_0.5_min_corr > 0, !(str_detect(study, '^AD')), !str_detect(study,'sQTL'), !str_detect(study,'PSI')) %>% select(study, region, chr, pos)  %>% merge(., gene_ref, by.x = 'region', by.y = 'gene_id')

## load gene list resources

In [221]:
gene_list <- fread('/data/resource//gene_list/AD_targets.csv')

In [255]:
nia_tb1 <- fread('/data/resource//gene_list//ADSP_NIAGADS_Table1.tsv', header = T)
nia_tb2 <- fread('/data/resource//gene_list//ADSP_NIAGADS_Table2.tsv', header = T)

In [256]:
colnames(nia_tb1) <- nia_tb1[1,] %>% as.character
nia_tb1 <- nia_tb1[-1,]

In [257]:
colnames(nia_tb2) <- nia_tb2[1,] %>% as.character
nia_tb2 <- nia_tb2[-1,]

In [258]:
nia_tb1 <- nia_tb1 %>% mutate(chr = paste0('chr', Chr), pos = `Location (hg38)` %>% as.numeric)

In [259]:
nia_tb2 %>% head

Number,Gene,Source,Location (hg38)
<chr>,<chr>,<chr>,<chr>
1,ABCA7,GVC,"chr19:1,039,997-1,065,572"
2,ABI3,GVC,"chr17:49,210,411-49,223,225"
3,APOE,GVC,"chr19:44,905,791-44,909,393"
4,APP,GVC,"chr21:25,880,550-26,171,128"
5,APBB3,Novikova,"chr5:140,558,268-140,564,781"
6,BIN1,GVC/Novikova,"chr2:127,048,023-127,107,288"


## annotate table 1 with xQTL flatten table

In [246]:
# nia_tb1_anno_all <- merge(nia_tb1, fla_tb_95_dis, by = c('chr', 'pos')) %>% merge(., gene_ref, by.x = 'region', by.y = 'gene_id')
nia_tb1_anno_nosTQL <- merge(nia_tb1, fla_tb_95_dis %>% filter(!str_detect(study,'sQTL'), !str_detect(study,'PSI')), by = c('chr', 'pos'))

In [238]:
nia_tb1_anno_nosTQL %>% pull(gene_name) %>% unique %>% length
nia_tb1 %>% pull(`Reported Gene/ Closest gene`) %>% unique %>% length

[1] 35

[1] 76

In [390]:
nia_tb1%>% filter(`Reported Gene/ Closest gene` == 'APP')

Number,Chr,Location (hg38),SNV,Reported Gene/ Closest gene,chr,pos
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
75,21,26101558,rs2154481,APP,chr21,26101558


In [394]:
fla_tb %>% filter(chr == 21, pos == 26101558)

variant_id,chr,pos,pip,maf,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,cs_coverage_0.95_min_corr,cs_coverage_0.7_min_corr,cs_coverage_0.5_min_corr,annotated_susie_cs,study,region,method,betahat,sebetahat
<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>


### some loci (variants) are annotated to different gene from NIAGADS Table 1

In [264]:
nia_tb1_anno_nosTQL <- nia_tb1_anno_nosTQL %>% 
                                group_by(`Reported Gene/ Closest gene`) %>% 
                                summarize(annotated_xQTL = paste(gene_name %>% unique, collapse =',' )) %>% left_join(., nia_tb1_anno_nosTQL) 

Joining with `by = join_by(`Reported Gene/ Closest gene`)`


In [265]:
nia_tb1_anno_nosTQL_same <- nia_tb1_anno_nosTQL %>% filter(`Reported Gene/ Closest gene` == annotated_xQTL)
nia_tb1_anno_nosTQL_debate <- nia_tb1_anno_nosTQL %>% filter(`Reported Gene/ Closest gene` != annotated_xQTL)

In [240]:
debate <- nia_tb1_anno_nosTQL %>% filter(`Reported Gene/ Closest gene` != annotated_xQTL)

In [343]:
nia_tb1_anno_nosTQL_debate1 <- nia_tb1_anno_nosTQL_debate %>%  mutate(Gene_name = `Reported Gene/ Closest gene` ,
                                                                      Projects_or_PI_interested = 'NIAGADS_1',
                                                                      NIAGADS_debate = annotated_xQTL) %>% 
select(Gene_name, Projects_or_PI_interested,NIAGADS_debate)
nia_tb1_anno_nosTQL_debate2 <- nia_tb1_anno_nosTQL_debate %>%  mutate(Gene_name = gene_name,
                                                                      Projects_or_PI_interested = 'NIAGADS_1',
                                                                      xQTL_debate = `Reported Gene/ Closest gene`) %>% 
select(Gene_name, Projects_or_PI_interested,xQTL_debate)

### format all gene list

In [398]:
gene_list <- gene_list %>% mutate(Gene_name = `Gene name`, Projects_or_PI_interested = `Target in project`) 

# nia_tb1_anno_nosTQL_anno <- nia_tb1_anno_nosTQL_anno %>% mutate(Gene_name = gene_name) # kept Table1's gene name for annotated 
nia_tb1_anno_nosTQL_same <- nia_tb1_anno_nosTQL_same %>% mutate(Gene_name = gene_name) %>% 
                    mutate(Projects_or_PI_interested = 'NIAGADS_1', xQTL_debate = 'consistent', NIAGADS_debate = 'consistent')
nia_tb1_noanno <- nia_tb1 %>% mutate(Gene_name = `Reported Gene/ Closest gene`) %>% 
                    filter(!(Gene_name %in% nia_tb1_anno_nosTQL$`Reported Gene/ Closest gene`)) %>% 
                    mutate(Projects_or_PI_interested = 'NIAGADS_1')

nia_tb2 <- nia_tb2 %>% mutate(Gene_name = Gene,Projects_or_PI_interested = 'NIAGADS_2') 

# debate <- debate %>% mutate(Projects_or_PI_interested = paste('Debate_to_NIAGADS_Table1:',`Reported Gene/ Closest gene`))
# debate <- debate %>% mutate(Projects_or_PI_interested = paste('Debate_to_NIAGADS_Table1'))

### combine existing genelist, 2 tables from NIAGADS and debate table together

In [399]:
long_list <- plyr::rbind.fill(gene_list, 
                              nia_tb1_anno_nosTQL_same,
                              nia_tb1_noanno, 
                              nia_tb1_anno_nosTQL_debate1,
                             nia_tb1_anno_nosTQL_debate2,
                             nia_tb2) %>% 
                group_by(Gene_name) %>% 
                summarize(Projects_or_PI_interested = paste(Projects_or_PI_interested %>% unique, collapse = ', '), NIAGADS_debate = NIAGADS_debate, xQTL_debate = xQTL_debate)  %>% distinct

Warning message:
"Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
i Please use `reframe()` instead.
i When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly."
`summarise()` has grouped output by 'Gene_name'. You can override using the
`.groups` argument.


In [400]:
# Remove the first row and handle duplicates based on conditions
long_list <- long_list[-1, ] %>%
  group_by(Gene_name, Projects_or_PI_interested) %>%
  # If a group has multiple rows, keep the one with non-NA values; 
  # if only one row exists in the group, keep it as is
  filter(if (n() > 1) row_number(desc(!is.na(NIAGADS_debate) | !is.na(xQTL_debate))) == 1 else TRUE) %>%
  ungroup()


In [418]:
#add LACTB
long_list <- rbind(long_list, data.frame(Gene_name = 'LACTB', Projects_or_PI_interested = 'xQTL main paper', NIAGADS_debate = NA, xQTL_debate = NA))

In [421]:
long_list %>% dim

[1] 157   4

### add one colume to document which study have min_corr_95 cs for each gene
exclude all sQTL

In [318]:
fla_tb_95_dis_noQTL_study <- fla_tb_95_dis_noQTL %>% group_by(gene_name) %>% summarize(study_xQTL = paste(study %>% unique, collapse = ', '))

In [412]:
fla_tb_95_dis_noQTL$study %>% str_extract(.,'^.+QTL') %>% unique 

[1] "MiGA_THA_eQTL"        "BM_22_MSBB_eQTL"      "Exc_DeJager_eQTL"    
 [4] "DLPFC_DeJager_eQTL"   "AC_DeJager_eQTL"      "Exc_mega_eQTL"       
 [7] "DLPFC_Bennett_pQTL"   "STARNET_eQTL"         "MiGA_GFM_eQTL"       
[10] "MiGA_SVZ_eQTL"        "Exc_Kellis_eQTL"      "Knight_eQTL"         
[13] "MiGA_GTS_eQTL"        "BM_10_MSBB_eQTL"      "BM_36_MSBB_eQTL"     
[16] "BM_44_MSBB_eQTL"      "PCC_DeJager_eQTL"     "Oli_Kellis_eQTL"     
[19] "MSBB_BM36_pQTL"       "Ast_DeJager_eQTL"     "Ast_mega_eQTL"       
[22] "Oli_mega_eQTL"        "Oli_DeJager_eQTL"     "OPC_DeJager_eQTL"    
[25] "OPC_Kellis_eQTL"      "OPC_mega_eQTL"        "Ast_Kellis_eQTL"     
[28] "monocyte_ROSMAP_eQTL" "Inh_DeJager_eQTL"     "Inh_Kellis_eQTL"     
[31] "Inh_mega_eQTL"        "Mic_Kellis_eQTL"      "DLPFC_Klein_gpQTL"   
[34] "Mic_13_Kellis_eQTL"   "Ast_10_Kellis_eQTL"   "Mic_mega_eQTL"       
[37] "Mic_12_Kellis_eQTL"   "Mic_DeJager_eQTL"     "10_MSBB_eQTL"

In [422]:
fla_tb_95_dis_noQTL_study_longlist <- fla_tb_95_dis_noQTL_study %>% filter(gene_name %in% long_list$Gene_name) %>% mutate(Gene_name = gene_name) %>% select(-gene_name)

In [423]:
fla_tb_95_dis_noQTL_study_longlist %>% filter(Gene_name == 'KAT8')

study_xQTL,Gene_name
<chr>,<chr>
"MiGA_GFM_eQTL, MiGA_SVZ_eQTL, MiGA_THA_eQTL, BM_10_MSBB_eQTL, BM_22_MSBB_eQTL, BM_36_MSBB_eQTL, Exc_DeJager_eQTL, Inh_DeJager_eQTL, DLPFC_DeJager_eQTL, PCC_DeJager_eQTL, AC_DeJager_eQTL, Exc_mega_eQTL, STARNET_eQTL_Mac",KAT8


In [428]:
long_list_study <- long_list %>% left_join(.,fla_tb_95_dis_noQTL_study_longlist) 
long_list_study %>% fwrite('2024Oct_gene_list_157.tsv', sep = '\t')

Joining with `by = join_by(Gene_name)`


In [429]:
long_list_study %>% filter(Gene_name == 'BCKDK')
long_list_study %>% filter(Gene_name == 'KAT8')

Gene_name,Projects_or_PI_interested,NIAGADS_debate,xQTL_debate,study_xQTL
<chr>,<chr>,<chr>,<chr>,<chr>
BCKDK,NIAGADS_1,"KAT8,ZNF843,PRSS36",NA,"MiGA_THA_eQTL, Inh_Kellis_eQTL, STARNET_eQTL_Mac"


Gene_name,Projects_or_PI_interested,NIAGADS_debate,xQTL_debate,study_xQTL
<chr>,<chr>,<chr>,<chr>,<chr>
KAT8,NIAGADS_1,NA,BCKDK,"MiGA_GFM_eQTL, MiGA_SVZ_eQTL, MiGA_THA_eQTL, BM_10_MSBB_eQTL, BM_22_MSBB_eQTL, BM_36_MSBB_eQTL, Exc_DeJager_eQTL, Inh_DeJager_eQTL, DLPFC_DeJager_eQTL, PCC_DeJager_eQTL, AC_DeJager_eQTL, Exc_mega_eQTL, STARNET_eQTL_Mac"


In [430]:
long_list_study %>% filter(Gene_name == 'LACTB')

Gene_name,Projects_or_PI_interested,NIAGADS_debate,xQTL_debate,study_xQTL
<chr>,<chr>,<chr>,<chr>,<chr>
LACTB,xQTL main paper,NA,NA,"Knight_eQTL_brain, MiGA_GFM_eQTL, MiGA_SVZ_eQTL, MiGA_THA_eQTL, BM_10_MSBB_eQTL, BM_22_MSBB_eQTL, BM_36_MSBB_eQTL, BM_44_MSBB_eQTL, MSBB_BM36_pQTL_chr15_ENSG00000103642_P83111, Mic_DeJager_eQTL, Oli_DeJager_eQTL, Exc_DeJager_eQTL, DLPFC_DeJager_eQTL, PCC_DeJager_eQTL, AC_DeJager_eQTL, Mic_Kellis_eQTL, Oli_Kellis_eQTL, Inh_Kellis_eQTL, Ast_mega_eQTL, Inh_mega_eQTL, Mic_mega_eQTL, Oli_mega_eQTL, DLPFC_Bennett_pQTL, STARNET_eQTL_Mac"


In [431]:
long_list_study 

Gene_name,Projects_or_PI_interested,NIAGADS_debate,xQTL_debate,study_xQTL
<chr>,<chr>,<chr>,<chr>,<chr>
ABCA1,NIAGADS_1,consistent,consistent,"MiGA_GTS_eQTL, MiGA_SVZ_eQTL, Exc_DeJager_eQTL, AC_DeJager_eQTL, Exc_Kellis_eQTL, Exc_mega_eQTL, Inh_mega_eQTL, Mic_mega_eQTL, monocyte_ROSMAP_eQTL"
ABCA7,"Shulman, NIAGADS_1, NIAGADS_2",NA,NA,"Knight_eQTL_brain, MiGA_GTS_eQTL, MiGA_SVZ_eQTL, BM_22_MSBB_eQTL, BM_44_MSBB_eQTL, Oli_DeJager_eQTL, DLPFC_DeJager_eQTL, PCC_DeJager_eQTL, AC_DeJager_eQTL, Oli_Kellis_eQTL, DLPFC_Bennett_pQTL, DLPFC_Klein_gpQTL_adjusted_gp_6964_Q8IZY2, DLPFC_Klein_gpQTL_adjusted_gp_6965_Q8IZY2, DLPFC_Klein_gpQTL_unadjusted_gp_6964_Q8IZY2, DLPFC_Klein_gpQTL_unadjusted_gp_6965_Q8IZY2, monocyte_ROSMAP_eQTL, STARNET_eQTL_Mac"
ABHD17C,NIAGADS_1,NA,CTSH,"MiGA_GTS_eQTL, MiGA_SVZ_eQTL, BM_22_MSBB_eQTL, Ast_DeJager_eQTL, DLPFC_DeJager_eQTL, AC_DeJager_eQTL, Ast_Kellis_eQTL, Ast_mega_eQTL, STARNET_eQTL_Mac"
ABI3,"NIAGADS_1, NIAGADS_2",NA,NA,"MiGA_GFM_eQTL, MiGA_GTS_eQTL, MiGA_SVZ_eQTL, BM_44_MSBB_eQTL, Mic_mega_eQTL, STARNET_eQTL_Mac"
ACE,"gpQTL, NIAGADS_1",consistent,consistent,"Knight_eQTL_brain, BM_22_MSBB_eQTL, BM_36_MSBB_eQTL, BM_44_MSBB_eQTL, MSBB_BM36_pQTL_chr17_ENSG00000159640_P12821, Exc_DeJager_eQTL, DLPFC_DeJager_eQTL, PCC_DeJager_eQTL, Exc_mega_eQTL, DLPFC_Bennett_pQTL, DLPFC_Klein_gpQTL_adjusted_gp_2630_P12821, DLPFC_Klein_gpQTL_adjusted_gp_2632_P12821, DLPFC_Klein_gpQTL_unadjusted_gp_2629_P12821, DLPFC_Klein_gpQTL_unadjusted_gp_2630_P12821, DLPFC_Klein_gpQTL_unadjusted_gp_2632_P12821"
ADAM10,"Shulman, NIAGADS_2",NA,NA,"MiGA_GTS_eQTL, MiGA_SVZ_eQTL, BM_36_MSBB_eQTL, Oli_DeJager_eQTL, Oli_Kellis_eQTL, Exc_Kellis_eQTL, Exc_mega_eQTL, Mic_mega_eQTL, Oli_mega_eQTL, DLPFC_Bennett_pQTL, DLPFC_Klein_gpQTL_adjusted_gp_0147_O14672, DLPFC_Klein_gpQTL_adjusted_gp_0154_O14672, DLPFC_Klein_gpQTL_adjusted_gp_0155_O14672, DLPFC_Klein_gpQTL_adjusted_gp_0156_O14672, DLPFC_Klein_gpQTL_adjusted_gp_0157_O14672, DLPFC_Klein_gpQTL_unadjusted_gp_0154_O14672, DLPFC_Klein_gpQTL_unadjusted_gp_0156_O14672, DLPFC_Klein_gpQTL_unadjusted_gp_0157_O14672, STARNET_eQTL_Mac"
ADAM17,"xQTL main paper, Shulman, NIAGADS_1",ITGB1BP1,NA,"Knight_eQTL_brain, MiGA_GTS_eQTL, MiGA_SVZ_eQTL, MiGA_THA_eQTL, BM_10_MSBB_eQTL, BM_22_MSBB_eQTL, BM_36_MSBB_eQTL, BM_44_MSBB_eQTL, Ast_DeJager_eQTL, Oli_DeJager_eQTL, Exc_DeJager_eQTL, Inh_DeJager_eQTL, DLPFC_DeJager_eQTL, PCC_DeJager_eQTL, AC_DeJager_eQTL, Ast_Kellis_eQTL, Oli_Kellis_eQTL, Exc_Kellis_eQTL, Ast_mega_eQTL, Exc_mega_eQTL, Inh_mega_eQTL, Mic_mega_eQTL, OPC_mega_eQTL, Oli_mega_eQTL, DLPFC_Klein_gpQTL_adjusted_gp_5127_P78536"
ADAMTS1,NIAGADS_1,NA,NA,"MiGA_GFM_eQTL, MiGA_GTS_eQTL, MiGA_SVZ_eQTL, MiGA_THA_eQTL"
AKAP9,Shulman,NA,NA,NA
